# Extras

In [ ]:
!pip install openai

## Que tanto estoy gastando?

Tenemos que [consultar la pagina](https://ai.google.dev/gemini-api/docs/pricing)  de cada proveedor y cada modelo

In [ ]:
from openai import OpenAI
from pydantic import BaseModel
from google.colab import userdata

#-----------
# todos los calculos los vamos a basar en los siguientes datos

tokens = {
    'entrada': 0,
    'salida': 0
}
model = "gemini-2.5-flash"
costo_por_token_entrada = 0.0000003
costo_por_token_salida =  0.0000003

def get_gasto_total():
  return {
      'costo' : (tokens['entrada'] * costo_por_token_entrada) + (tokens['salida'] * costo_por_token_salida),
      'tokens': tokens['entrada'] + tokens['salida']
  }
#-----------

# en este caso la funcion de inferencia nos permite pasar un prompt y un
# modelo pydantic para hacer nuestra salida estructurada.
def inferencia(content, pydantic_model):
  client = OpenAI(
      api_key=userdata.get('GOOGLE_API_KEY'),
      base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
      max_retries=2 #numero de reintentos
  )

  response = client.beta.chat.completions.parse(
      model=model,
      messages=[
          {
              "role": "user",
              "content": content
          }
      ],
      response_format=pydantic_model
  )
  message = response.choices[0].message
  tokens['entrada'] = tokens['entrada'] + response.usage.prompt_tokens
  tokens['salida'] = tokens['salida'] + response.usage.completion_tokens
  if message.parsed:
    return message.parsed
  else:
    return message.choices[0].message.content

## Extraer informacion de personas

In [ ]:
class PersonaResponse(BaseModel):
  nombre: str
  edad: int
  ciudad: str

print(inferencia("Mi nombre es Cosme Zamudio, tengo 40 años y vivo en Mazatlan Sinaloa, Mexico.", PersonaResponse))
print(inferencia("Mi nombre es Juan Ortega, tengo 35 años y vivo en Culiacan Sinaloa, Mexico.", PersonaResponse))
print(inferencia("Mi nombre es Alan Garzon, tengo 29 años y vivo en Ciudad de Mexico.", PersonaResponse))

## Gasto hasta ahora

In [ ]:
get_gasto_total()

## Extraccion imagen + pydantic + costos


In [ ]:
import requests
import base64

# helper para convertir el contenido de alguna url a base64
def url_to_base64(url):
    response = requests.get(url)
    response.raise_for_status()
    encoded = base64.b64encode(response.content).decode('utf-8')
    return encoded

#que informacion queremos extraer de esta imagen?

class CalzadoResponse(BaseModel):
  color: str
  color_secundario: str
  tipo_calzado: str

base64img = url_to_base64("https://storage.googleapis.com/tallerdp_publico/img/1000100-0001V1.jpg")
content = [
  {"type": "text", "text": "Del siguiente calzado extrae el color (primario), el color secundario y el tipo de calzado (tenis, zapatos, sandalias, etc)"},
  {"type": "image_url", "image_url": { "url" : f"data:image/jpeg;base64,{base64img}"}}
]
print(inferencia(content, CalzadoResponse))

base64img = url_to_base64("https://storage.googleapis.com/tallerdp_publico/img/1020834-0000V1.jpg")
content = [
  {"type": "text", "text": "Del siguiente calzado extrae el color (primario), el color secundario y el tipo de calzado (tenis, zapatos, sandalias, etc)"},
  {"type": "image_url", "image_url": { "url" : f"data:image/jpeg;base64,{base64img}"}}
]
print(inferencia(content, CalzadoResponse))


base64img = url_to_base64("https://storage.googleapis.com/tallerdp_publico/img/1021611-0303V1.jpg")
content = [
  {"type": "text", "text": "Del siguiente calzado extrae el color (primario), el color secundario y el tipo de calzado (tenis, zapatos, sandalias, etc)"},
  {"type": "image_url", "image_url": { "url" : f"data:image/jpeg;base64,{base64img}"}}
]
print(inferencia(content, CalzadoResponse))


print(get_gasto_total())


## Formato PDF

El formato más utilizado para extraer información es el PDF, sin embargo, debemos tener en cuenta que los PDFs pueden contener texto, una imagen que se asemeje a texto o un texto.


In [ ]:
!pip install --upgrade pymupdf
import fitz  #comentar sobre licencia
from io import StringIO

def pdf_to_text(url):
    response = requests.get(url)
    response.raise_for_status()
    path = "temp.pdf"
    with open(path, "wb") as f:
        f.write(response.content)


    doc = fitz.open(path)
    sb = StringIO()

    for i, page in enumerate(doc, 1):
        sb.write(f"texto de pagina {i:02d}:\n```\n")
        sb.write(page.get_text())
        sb.write("```\n\n")
    return sb.getvalue()

print(pdf_to_text("https://storage.googleapis.com/tallerdp_publico/pdf/pdf_texto.pdf"))

In [ ]:
# Vamos a obtener las personas, el cargo y el numero de pagina donde se encuentra en el PDF
from typing import List

class Persona(BaseModel):
    nombre: str
    cargo: str
    pagina: int

class PersonasExtraidas(BaseModel):
    personas: List[Persona]


prompt = f"""
Del siguiente contenido de un PDF

{pdf_to_text("https://storage.googleapis.com/tallerdp_publico/pdf/pdf_texto.pdf")}

Extrae una lista de las personas con los siguientes datos:
- Nombre de la persona
- Cargo que ocupa
- Pagina donde se encuentra
"""

print(inferencia(prompt, PersonasExtraidas))
print(get_gasto_total())

### PDF como una imagen

Hay casos en los que necesitamos trabajar con información visual del PDF, por ejemplo, preguntar acerca de la posición de algún elemento, logotipos o imágenes dentro del documento.

Para esto, lo ideal es renderizar la página y tratarla como una imagen.

In [ ]:
from io import BytesIO
def pdf_url_to_base64(url):
    response = requests.get(url)
    response.raise_for_status()

    pdf_stream = BytesIO(response.content)
    doc = fitz.open(stream=pdf_stream, filetype="pdf")
    page = doc.load_page(0) #solo renderizamos la primera pagina
    pix = page.get_pixmap(dpi=150) #es recomendable trabajr con 300 cuando hablamos de planos


    img_bytes = pix.tobytes("png")
    encoded = base64.b64encode(img_bytes).decode("utf-8")
    return encoded


# vamos a extraer la lista de valvulas mencionadas en el plano
class ValvulasExtraidas(BaseModel):
    valvulas: List[str]



base64img = pdf_url_to_base64("https://storage.googleapis.com/tallerdp_publico/pdf/pdf_imagen.pdf")
content = [
  {"type": "text", "text": "Del siguiente plano, extrae todos los tipos de valcular mencionadas"},
  {"type": "image_url", "image_url": { "url" : f"data:image/png;base64,{base64img}"}}
]
print(inferencia(content, ValvulasExtraidas))
print(get_gasto_total())

## Typings y Json API

[Internamente](https://github.com/openai/openai-python/blob/e68921654125ae733aac00c683b504bc89856df2/src/openai/lib/_parsing/_completions.py#L232) OpenAI convierte cualquier modelo de pydantic en Json API, usando un metodo interno.

Otros metodos para generar json API:


*   [Google AI Studio](https://ai.dev/)
*   [OpenAI Playground](https://platform.openai.com/playground/prompts)



In [ ]:
import json

print("PersonaResponse:")
print(json.dumps(PersonaResponse.model_json_schema(), indent=2))

print("CalzadoResponse:")
print(json.dumps(CalzadoResponse.model_json_schema(), indent=2))

print("PersonasExtraidas:")
print(json.dumps(PersonasExtraidas.model_json_schema(), indent=2))

print("ValvulasExtraidas:")
print(json.dumps(ValvulasExtraidas.model_json_schema(), indent=2))

## Otros proveedores



### OpenRouter

[OpenRouter](https://openrouter.ai/) ofrece una API unificada que te da acceso a cientos de modelos de IA a través de un solo endpoint, manejando automáticamente las alternativas y seleccionando las opciones más rentables.

In [ ]:
def inferencia_openrouter(openrouter_model, content, pydantic_model):
  client = OpenAI(
      api_key=userdata.get('OPENROUTER_API_KEY'),
      base_url="https://openrouter.ai/api/v1",
      max_retries=2 #numero de reintentos
  )

  response = client.beta.chat.completions.parse(
      model=openrouter_model,
      messages=[
          {
              "role": "user",
              "content": content
          }
      ],
      response_format=pydantic_model
  )
  message = response.choices[0].message
  if message.parsed:
    return message.parsed
  else:
    return message.choices[0].message.content

Que tan facil es cambiar de modelo con openrouter?

In [ ]:
print(inferencia_openrouter("meta-llama/llama-3.3-8b-instruct:free", "Mi nombre es Cosme Zamudio, tengo 40 años y vivo en Mazatlan Sinaloa, Mexico.", PersonaResponse))
print(inferencia_openrouter("mistralai/mistral-small-3.2-24b-instruct:free", "Mi nombre es Juan Ortega, tengo 35 años y vivo en Culiacan Sinaloa, Mexico.", PersonaResponse))
print(inferencia_openrouter("meta-llama/llama-4-maverick:free", "Mi nombre es Alan Garzon, tengo 29 años y vivo en Ciudad de Mexico.", PersonaResponse))



### Together AI

Together es una plataforma que ofrece acceso directo  a modelos de lenguaje open-source como LLaMA y Mistral, ideal para quienes buscan rendimiento con pesos abiertos y posibilidad de fine-tuning; en cambio, OpenRouter actúa como un agregador que permite usar una sola API para acceder a múltiples modelos tanto open como cerrados (como GPT-4 o Claude) desde distintos proveedores, facilitando la comparación y el cambio entre ellos sin modificar el código.

In [ ]:
def inferencia_together(together_model, content, pydantic_model):
  client = OpenAI(
      api_key=userdata.get('TOGETHERAI_API_KEY'),
      base_url="https://api.together.xyz/v1",
      max_retries=2 #numero de reintentos
  )

  response = client.beta.chat.completions.parse(
      model=together_model,
      messages=[
          {
              "role": "user",
              "content": content
          }
      ],
      response_format=pydantic_model
  )
  message = response.choices[0].message
  if message.parsed:
    return message.parsed
  else:
    return message.choices[0].message.content


print(inferencia_together("meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo", "Mi nombre es Alan Garzon, tengo 29 años y vivo en Ciudad de Mexico.", PersonaResponse))

Podemos abstraer nuestra funcion de inferencia de la siguiente manera:


In [ ]:
def infer(content, pydantic_model, model = "gpt-4.1-mini", api_key=userdata.get('OPENAI_API_KEY'), base_url='https://api.openai.com/v1'):
  client = OpenAI(
      api_key=api_key,
      base_url=base_url,
      max_retries=2
  )

  response = client.beta.chat.completions.parse(
      model=model,
      messages=[
          {
              "role": "user",
              "content": content
          }
      ],
      response_format=pydantic_model
  )
  message = response.choices[0].message
  if message.parsed:
    return message.parsed
  else:
    return message.choices[0].message.content

#OPENAI
print(infer("Mi nombre es Alan Garzon, tengo 29 años y vivo en Ciudad de Mexico.", PersonaResponse))
#TOGETHERAI
print(infer("Mi nombre es Alan Garzon, tengo 29 años y vivo en Ciudad de Mexico.",
            PersonaResponse,
            model = 'meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo',
            api_key=userdata.get('TOGETHERAI_API_KEY'),
            base_url='https://api.together.xyz/v1'))
#CEREBRAS
print(infer("Mi nombre es Alan Garzon, tengo 29 años y vivo en Ciudad de Mexico.",
            PersonaResponse,
            model = 'llama-4-scout-17b-16e-instruct',
            api_key=userdata.get('CEREBRAS_API_KEY'),
            base_url='https://api.cerebras.ai/v1'))

## Ejercicio Final